# Import Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import progressbar

# Grabbing URL

In [2]:
allPlayersURL = "https://www.basketball-reference.com/players/a"
allPlayersPageRequest = requests.get(allPlayersURL)

## Parse URL Page

In [3]:
page_soup = BeautifulSoup(allPlayersPageRequest.content, "html.parser")
table = page_soup.find("table")


## Grabbing NBA Data

In [4]:
count = 0
playersDB = []
if table:
    table_body = table.find("tbody")
    for row in table_body:
        table_body.find_all("tr")
        player_url = row.find("a")
        try:
            # all data for all players uniform across database
            cells = row.findAll("td")
            active_from = int(cells[0].text)
        except (AttributeError):
            continue
        if (active_from > 2009):
            try:
                count += 1
                player_names = player_url.text
                active_to = int(cells[1].text)
                position = cells[2].text
                college = "Pittsburgh" if cells[6].text == "Pitt" else cells[6].text
                player_entry = {
                    'name': player_names,
                    'active_from': active_from,
                    'active_to': active_to,
                    'position': position,
                    'college': college,
                    'url': "https://www.basketball-reference.com"+player_url['href']
                }
                playersDB.append(player_entry)
            except (AttributeError):
                continue

## Grabbing College Data

In [6]:
allCollegePlayersURL = "https://www.sports-reference.com/cbb/players/a-index.html"
allCollegePlayersPageRequest = requests.get(allCollegePlayersURL)
college_page_soup = BeautifulSoup(allCollegePlayersPageRequest.content, "html.parser")

college_div = college_page_soup.find("div", id="content")
college_list_header = college_div.find_next("h2").find_next("p")
college_total_list = college_list_header.find_next_siblings("div")

college_duration = ""
playersDBWithCollege = []
# bar = progressbar.ProgressBar(max_value=len(playersDB))

for i in playersDB:
    count += 1
    if (i["college"] != ""):
        for j in college_total_list:
            short_list = j.find_all("p")
            for x in short_list:
                if ((x.find_next("a").text == i["name"]) and (x.find_next("small").find_next("a").text == i["college"])):
                    year = x.find_next("small")
                    print(x.find_next("a").text) # Player Name
                    college_duration = year.find(text=True, recursive=False)
                    print(year.find(text=True, recursive=False))
                    print(x.find_next("small").find_next("a").text) # Player College
    player_entry_college = {
        'name': i["name"],
        'active_from': i["active_from"],
        'active_to': i["active_to"],
        'position': i["position"],
        'college': i["college"],
        'college_duration': college_duration,
        'url': i["url"]
    }
    playersDBWithCollege.append(player_entry_college)
    college_duration = ""
    # bar.update(count)

Precious Achiuwa
(2020-2020) 
Memphis
Quincy Acy
(2009-2012) 
Baylor
Jaylen Adams
(2015-2018) 
St. Bonaventure
Jordan Adams
(2013-2014) 
UCLA
Steven Adams
(2013-2013) 
Pittsburgh
Bam Adebayo
(2017-2017) 
Kentucky
Deng Adel
(2016-2018) 
Louisville
DeVaughn Akoon-Purcell
(2015-2016) 
Illinois State
Solomon Alabi
(2008-2010) 
Florida State
Santi Aldama
(2020-2021) 
Loyola (MD)
Cole Aldrich
(2008-2010) 
Kansas
Cliff Alexander
(2015-2015) 
Kansas
Kyle Alexander
(2016-2019) 
Tennessee
Ty-Shon Alexander
(2018-2020) 
Creighton
Nickeil Alexander-Walker
(2018-2019) 
Virginia Tech
Rawle Alkins
(2017-2018) 
Arizona
Grayson Allen
(2015-2018) 
Duke
Jarrett Allen
(2017-2017) 
Texas
Kadeem Allen
(2016-2017) 
Arizona
Lavoy Allen
(2008-2011) 
Temple
Jose Alvarado
(2018-2021) 
Georgia Tech
Al-Farouq Aminu
(2009-2010) 
Wake Forest
Antonio Anderson
(2006-2009) 
Memphis
James Anderson
(2008-2010) 
Oklahoma State
Justin Anderson
(2013-2015) 
Virginia
Kyle Anderson
(2013-2014) 
UCLA
Ike Anigbogu
(2017-2017) 


## Create DF

In [7]:
df = pd.DataFrame(playersDBWithCollege)

## Output df to csv

In [8]:
df.to_csv("playerData.csv", encoding="utf-8", index_label="index")